# Assess generalizability of the model by using drug dose curve data

**NOTE:** For assess generalizability based on a drug dose response, we will be using Plates 1 and 2, split by heart number (all failing hearts).

## Import libraries

In [1]:
import pathlib
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import seaborn as sns
from joblib import load
from sklearn.metrics import precision_recall_curve

sys.path.append("../utils")
from eval_utils import generate_confusion_matrix_df, generate_f1_score_df
from training_utils import get_X_y_data

## Set paths and variables

In [2]:
# Directory with plate datasets
data_dir = pathlib.Path("../3.process_cfret_features/data/single_cell_profiles")

# Directory with models
models_dir = pathlib.Path("./models")

# Directory for model figures output
fig_dir = pathlib.Path("./figures")
fig_dir.mkdir(exist_ok=True)

# Directory for probability data to be saved
prob_dir = pathlib.Path("./prob_data")
prob_dir.mkdir(exist_ok=True)

# Load in each model individually
final_model = load(
    pathlib.Path(f"{models_dir}/log_reg_fs_plate_4_final_downsample.joblib")
)
shuffled_model = load(
    pathlib.Path(f"{models_dir}/log_reg_fs_plate_4_shuffled_downsample.joblib")
)

## Load in Plate 4 fs data to extract column names to filter from the other plates

In [3]:
# Load in Plate 4 normalized feature selected data metadata (used with model) to get the feature columns to filter the plate data
parquet_metadata = pq.read_metadata(
    pathlib.Path(f"{data_dir}/localhost231120090001_sc_feature_selected.parquet")
)

# Get the column names from the metadata
all_column_names = parquet_metadata.schema.names

# Filter out the column names that start with "Metadata_"
model_column_names = [
    col for col in all_column_names if not col.startswith("Metadata_")
]

print(len(model_column_names))
print(model_column_names)

638
['Cytoplasm_AreaShape_Compactness', 'Cytoplasm_AreaShape_Eccentricity', 'Cytoplasm_AreaShape_Extent', 'Cytoplasm_AreaShape_FormFactor', 'Cytoplasm_AreaShape_MajorAxisLength', 'Cytoplasm_AreaShape_MinorAxisLength', 'Cytoplasm_AreaShape_Perimeter', 'Cytoplasm_AreaShape_Solidity', 'Cytoplasm_AreaShape_Zernike_0_0', 'Cytoplasm_AreaShape_Zernike_1_1', 'Cytoplasm_AreaShape_Zernike_2_0', 'Cytoplasm_AreaShape_Zernike_2_2', 'Cytoplasm_AreaShape_Zernike_3_1', 'Cytoplasm_AreaShape_Zernike_3_3', 'Cytoplasm_AreaShape_Zernike_4_0', 'Cytoplasm_AreaShape_Zernike_4_2', 'Cytoplasm_AreaShape_Zernike_5_1', 'Cytoplasm_AreaShape_Zernike_5_3', 'Cytoplasm_AreaShape_Zernike_6_0', 'Cytoplasm_AreaShape_Zernike_6_2', 'Cytoplasm_AreaShape_Zernike_6_4', 'Cytoplasm_AreaShape_Zernike_7_1', 'Cytoplasm_AreaShape_Zernike_7_3', 'Cytoplasm_AreaShape_Zernike_8_0', 'Cytoplasm_AreaShape_Zernike_8_2', 'Cytoplasm_AreaShape_Zernike_8_4', 'Cytoplasm_AreaShape_Zernike_8_6', 'Cytoplasm_AreaShape_Zernike_9_1', 'Cytoplasm_AreaSh

## Load in Plates 1 and 2, concat vertically, and drop any rows where there are NaNs in the feature columns from the model

In [4]:
# Load in Plate 1 and 2 data -> concat
plate_1_df = pd.read_parquet(
    pathlib.Path(f"{data_dir}/localhost220512140003_KK22-05-198_sc_normalized.parquet")
)
plate_2_df = pd.read_parquet(
    pathlib.Path(
        f"{data_dir}/localhost220513100001_KK22-05-198_FactinAdjusted_sc_normalized.parquet"
    )
)

# Concat separate parts of the same plate together
concatenated_df = pd.concat([plate_1_df, plate_2_df], axis=0)

# Drop rows with NaN values in feature columns that the model uses
concatenated_df = concatenated_df.dropna(subset=model_column_names)

print(concatenated_df.shape)
concatenated_df.head()

(41066, 2022)


,Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_treatment,Metadata_dose,Metadata_dose_unit,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,...,Nuclei_Texture_Variance_Hoechst_3_02_256,Nuclei_Texture_Variance_Hoechst_3_03_256,Nuclei_Texture_Variance_Mitochondria_3_00_256,Nuclei_Texture_Variance_Mitochondria_3_01_256,Nuclei_Texture_Variance_Mitochondria_3_02_256,Nuclei_Texture_Variance_Mitochondria_3_03_256,Nuclei_Texture_Variance_PM_3_00_256,Nuclei_Texture_Variance_PM_3_01_256,Nuclei_Texture_Variance_PM_3_02_256,Nuclei_Texture_Variance_PM_3_03_256
0,A,1,3,drug_x,5.0,uM,220.491103,103.878114,238.995447,104.179514,...,-0.106047,-0.107179,-0.389969,-0.387109,-0.394170,-0.388847,-0.247949,-0.246999,-0.248843,-0.247074
1,A,1,3,drug_x,5.0,uM,180.302253,171.314685,199.191002,189.827676,...,-0.218297,-0.230013,-0.398917,-0.396269,-0.401236,-0.395099,-0.213771,-0.216189,-0.215117,-0.209354
2,A,1,3,drug_x,5.0,uM,765.912882,208.396664,777.621076,212.232104,...,-0.026298,-0.032182,-0.393256,-0.389578,-0.394368,-0.390305,-0.234386,-0.234441,-0.236695,-0.234486
3,A,1,3,drug_x,5.0,uM,377.832677,247.411417,363.699578,247.405714,...,0.075800,0.067021,-0.105392,-0.063613,-0.073499,-0.116253,0.333718,0.364179,0.340308,0.311893
4,A,1,3,drug_x,5.0,uM,533.028105,256.360834,550.957839,243.532560,...,-0.150196,-0.152180,-0.396738,-0.392573,-0.397565,-0.391232,-0.247228,-0.247879,-0.247876,-0.246180


## Filter the concat data to only include metadata and filtered feature columns

In [5]:
# Extract metadata columns from the plate
metadata_columns = [col for col in concatenated_df.columns if col.startswith("Metadata_")]

# Extract feature columns that don't start with "Metadata_"
feature_columns = [col for col in concatenated_df.columns if not col.startswith("Metadata_")]

# Filter columns in data frame to only include those in the model
filtered_feature_columns = [
    col for col in concatenated_df.columns if col in model_column_names
]

# Filter the DataFrame to keep only the desired columns
concatenated_df = concatenated_df[metadata_columns + filtered_feature_columns]

concatenated_df

,Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_treatment,Metadata_dose,Metadata_dose_unit,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,...,Nuclei_Texture_InverseDifferenceMoment_ER_3_01_256,Nuclei_Texture_InverseDifferenceMoment_ER_3_03_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_00_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_02_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_01_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_03_256,Nuclei_Texture_SumVariance_ER_3_03_256,Nuclei_Texture_SumVariance_Hoechst_3_03_256,Nuclei_Texture_SumVariance_Mitochondria_3_03_256,Nuclei_Texture_SumVariance_PM_3_01_256
0,A,1,3,drug_x,5.0,uM,220.491103,103.878114,238.995447,104.179514,...,0.765945,0.147124,1.905393,1.442735,0.801047,0.559833,-0.311096,-0.148883,-0.365400,-0.236899
1,A,1,3,drug_x,5.0,uM,180.302253,171.314685,199.191002,189.827676,...,0.412043,0.787652,1.611779,1.464592,0.831427,0.977141,-0.271848,-0.224770,-0.373039,-0.203665
2,A,1,3,drug_x,5.0,uM,765.912882,208.396664,777.621076,212.232104,...,1.004796,1.267602,1.603289,1.665712,0.807725,0.782182,-0.303927,-0.086479,-0.367481,-0.222132
3,A,1,3,drug_x,5.0,uM,377.832677,247.411417,363.699578,247.405714,...,0.523911,0.073946,0.847132,0.678840,-0.523384,-0.032206,-0.122677,-0.030187,-0.107171,0.320405
4,A,1,3,drug_x,5.0,uM,533.028105,256.360834,550.957839,243.532560,...,0.873611,0.367545,1.756592,0.937868,0.817433,0.837432,-0.299750,-0.165007,-0.370908,-0.238309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9521,H,12,9,DMSO,0.0,None,458.842771,507.345103,434.956106,527.094012,...,0.346245,-0.183756,0.979271,0.670610,0.310861,0.065047,-0.229127,-0.175570,-0.175362,-0.166514
9522,H,12,9,DMSO,0.0,None,472.660478,710.522026,434.594208,727.349028,...,-0.339112,-0.291470,0.903185,0.635543,0.221283,0.268469,0.299853,-0.164022,-0.161743,-0.080162
9523,H,12,9,DMSO,0.0,None,543.896603,633.143772,562.540248,608.033659,...,-0.183275,-0.090061,0.786796,-0.645665,-0.145998,-0.276909,-0.120810,-0.265058,0.019118,-0.015119
9524,H,12,9,DMSO,0.0,None,698.432695,855.316986,723.824246,843.527927,...,-0.738812,-0.642565,-0.152188,-0.406818,0.039585,0.003944,0.251274,-0.259491,-0.104783,-0.090674


## Create a dictionary with concat dataframe splitting the data by the heart number

In [6]:
# Split the plate data into different data frames with different data for applying the model to

# Define a dictionary
plate_3_dfs_dict = {}

# Filter the DataFrame to a data frame per treatment
three_df = concatenated_df[concatenated_df["Metadata_heart_number"] == 3]
eight_df = concatenated_df[concatenated_df["Metadata_heart_number"] == 8]
nine_df = concatenated_df[concatenated_df["Metadata_heart_number"] == 9]

# Add each DataFrame to the dictionary with a corresponding key
plate_3_dfs_dict["heart_3"] = {"data_df": three_df}
plate_3_dfs_dict["heart_8"] = {"data_df": eight_df}
plate_3_dfs_dict["heart_9"] = {"data_df": nine_df}

## Extract final model predicted probabilities for each heart number

In [7]:
# Create an empty DataFrame to store the results
combined_prob_df = pd.DataFrame()

for model_path in models_dir.iterdir():
    model_type = model_path.stem.split("_")[5]  # Get the model type

    for data, info in plate_3_dfs_dict.items():
        # Ensure that the file is named the correct data split
        data_split = data
        print(f"Extracting {model_type} probabilities from {data} data...")

        # Load in model to apply to datasets
        model = load(model_path)

        # Load in label encoder
        le = load(
            pathlib.Path("./encoder_results/label_encoder_log_reg_fs_plate_4.joblib")
        )

        # Get unique cell types and their corresponding encoded values
        unique_labels = le.classes_
        encoded_values = le.transform(unique_labels)

        # Create a dictionary mapping encoded values to original labels
        label_dict = dict(zip(encoded_values, unique_labels))

        # Load in data frame associated with the data split
        data_df = info["data_df"].reset_index(drop=True)

        # Load in X data to get predicted probabilities
        X, _ = get_X_y_data(df=data_df, label="Metadata_heart_number")

        # Predict class probabilities for morphology feature data
        predicted_probs = model.predict_proba(X)

        # Storing probabilities in a pandas DataFrame
        prob_df = pd.DataFrame(predicted_probs, columns=model.classes_)

        # Update column names in prob_df using the dictionary and add suffix "_probas"
        prob_df.columns = [label_dict[col] + "_probas" for col in prob_df.columns]

        # Add a new column called predicted_label for each row
        prob_df["predicted_label"] = prob_df.apply(
            lambda row: row.idxmax()[:-7], axis=1
        )

        # Select metadata columns from the data
        metadata_columns = data_df.filter(like="Metadata_")

        # Combine metadata columns with predicted probabilities DataFrame based on index
        prob_df = prob_df.join(metadata_columns)

        # Add a new column for model_type
        prob_df["model_type"] = model_type

        # Append the probability DataFrame to the combined DataFrame
        combined_prob_df = pd.concat([combined_prob_df, prob_df], ignore_index=True)

# Save combined prob data
combined_prob_df.to_csv(f"{prob_dir}/combined_plates_1_2_predicted_proba.csv", index=False)

Extracting shuffled probabilities from heart_3 data...
Extracting shuffled probabilities from heart_8 data...
Extracting shuffled probabilities from heart_9 data...
Extracting final probabilities from heart_3 data...
Extracting final probabilities from heart_8 data...
Extracting final probabilities from heart_9 data...
